<!-- Styling for the main heading -->
<h1 style="color: red; text-align: center;">Tactical Analysis by: The Modern Ball-Playing Goalkeeper: How André Onana Could Elevate Manchester United</h1>

<!-- Styling for the subheading -->
<h2 style="color: red; text-align: center;">Unlocking the Power of Passing: A Comprehensive PCA and Clustering Analysis of Goalkeeper Impact on Team Performance</h2>


### Importing required librarys

In [1]:
import pandas as pd
import numpy as np
warnings.filterwarnings("ignore")

NameError: name 'warnings' is not defined

In [ ]:
df= pd.read_csv('Top5leaguepassstats2021-22.csv')
df

In [ ]:
df.info()

### Taking club's league position from another data

In [ ]:
standings=pd.read_excel('top5league_stats.xlsx')
standings=standings.iloc[126:224,:]
standings

In [ ]:
data=standings.reset_index(drop=True)
l=['Rk','Squad','Country','LgRk','MP','W','D','L','GF','GA','GD','Pts','Pts/MP','xG','xGA','xGD','xGD/90','Attendance','Top Team Scorer','Goalkeeper']
data.columns=l
data

In [ ]:
# converting numarical coloumns to float 
x=['LgRk','MP','W','D','L','GF','GA','GD','Pts','Pts/MP','xG','xGA','xGD','xGD/90','Attendance']
for i in x:
    data[i]=data[i].astype('float')
    data[i]=data[i].round(decimals=2)

In [ ]:
# convert stats per 90 min
x=[ 'Total_Cmp',
       'Total_Att', 'TotDist', 'Total_PrgDist', 'short_Cmp', 'short_Att', 'medium_Cmp', 'medium_Att',
        'long_Cmp', 'long_Att', 'Assist', 'xAG', 'xA', 'A-xAG', 'KeyPass', '1/3',
       'PPA', 'CrsPA', 'PrgP']

for i in x:
    df[i]=df[i]/df['90s']

In [ ]:
data['Squad'] = data['Squad'].str.replace('Club Crest ', '')
data['Squad'].nunique()



In [ ]:
df['Squad'].nunique()

In [ ]:
# concating df with league position

x=data[['Squad','LgRk']]
merged_df = pd.merge(df, x, on='Squad',how='left')

merged_df

In [ ]:
# Taking GK data
gk=merged_df[merged_df['Pos']=='GK']
gk

In [ ]:
# Filtering GK Who played more than 17 games out of 38 league games
gk=gk[gk['90s']>17]
gk=gk.reset_index(drop=True)
gk

In [ ]:
# Filtering GK having Top6 and Bottom6 position in league table
gk['LgRk'] = gk['LgRk'].apply(pd.to_numeric, errors='coerce').round().astype(pd.Int32Dtype())
gk

# Create a boolean mask to filter the rows
mask = (gk['LgRk'] < 7) | (gk['LgRk'] > 14)

# Apply the mask to the DataFrame to remove the rows
gk = gk[mask]
gk

In [ ]:
# Labeling GK belonging to Top6 or Bottom6 
gk['LgRk'] = gk['LgRk'].astype('str')
# Define a dictionary to specify replacements
replacement_dict = {'15': 'Bottom6', '16': 'Bottom6', '17': 'Bottom6', '18': 'Bottom6', '19': 'Bottom6', '20': 'Bottom6',
                   '1': 'Top6', '2': 'Top6', '3': 'Top6', '4': 'Top6', '5': 'Top6', '6': 'Top6',}

# Use the replace method to replace values in the 'LgRk' column
gk['LgRk'] = gk['LgRk'].replace(replacement_dict)
gk=gk[[ 'Player', 'Nation', 'Squad', 'league', 'Age', 'Total_Cmp', 'Total_Att', 'Total_Cmp%', 'TotDist',
       'Total_PrgDist', 'short_Cmp', 'short_Att', 'short_Cmp%', 'medium_Cmp',
       'medium_Att', 'medium_Cmp%', 'long_Cmp', 'long_Att', 'long_Cmp%','PrgP', 'LgRk']]
# Now, the 'LgRk' column in 'gk' DataFrame will have the specified replacements
gk

In [ ]:
#Removing French league from the dataset to avoid too many points from upcoming scatterplot
top_bottom_6=gk[~gk['league'].isin( ['fr Ligue 1'])]
top_bottom_6=top_bottom_6.reset_index()
top_bottom_6

In [ ]:
top_bottom_6=top_bottom_6.drop('league',axis=1)

In [ ]:
# Identify and round columns with numeric data types
numeric_column = top_bottom_6.select_dtypes(include=['int64', 'float64'])
top_bottom_6[numeric_column.columns] = numeric_column.round(decimals=2)
top_bottom_6

## How Total_Cmp% vs long_Cmp% changes with each goalkeepers

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 8))
scatterplot = sns.scatterplot(data=top_bottom_6, x='Total_Cmp%', y='long_Cmp%', hue_order=['Squad','LgRk'])

# Set the y-axis limits to the minimum value of the data
plt.ylim(bottom=top_bottom_6['long_Cmp%'].min() - 1)

# Annotate the points with squad labels
for line in range(0, top_bottom_6.shape[0]):
    scatterplot.text(top_bottom_6['Total_Cmp%'][line] + 0.01, top_bottom_6['long_Cmp%'][line], 
                     top_bottom_6['Squad'][line], horizontalalignment='left', 
                     size='medium', color='black', weight='semibold')
plt.savefig('Total_Cmp%_long_Cmp%_scatterplot.png', bbox_inches='tight', pad_inches=0.05, dpi=300)
plt.show()


## Check whether top6 and bottom6 clubs are diffrentiate each other

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 8))
scatterplot = sns.scatterplot(data=top_bottom_6, x='Total_Cmp%', y='long_Cmp%', hue='LgRk')

# Set the y-axis limits to the minimum value of the data
plt.ylim(bottom=top_bottom_6['long_Cmp%'].min() - 1)

# Annotate the points with squad labels
for line in range(0, top_bottom_6.shape[0]):
    scatterplot.text(top_bottom_6['Total_Cmp%'][line] + 0.01, top_bottom_6['long_Cmp%'][line], 
                     top_bottom_6['Squad'][line], horizontalalignment='left', 
                     size='medium', color='black', weight='semibold')

plt.show()

- We can see that Top6 and bottom6 club Goalkeepers are diffrentiated each other. But there is a mixup between the points

## Let's add more features
- If we want to include more passing-related features like 'Total_Cmp,' 'Total_Att,' 'Total_Cmp%,' 'TotDist,' 'Total_PrgDist,' 'short_Cmp,' 'short_Att,' 'short_Cmp%,' 'medium_Cmp,' 'medium_Att,' 'medium_Cmp%,' 'long_Cmp,' 'long_Att,' 'long_Cmp%,' and 'PrgP,' it can be quite complex. These variables often show strong connections and duplication because they represent hidden aspects of the data.

## In the Data Science field, a lot of techniques exist for resolving this problem and they go under the name of dimensionality reduction. Among them, the Principal Component Analysis (PCA) is probably the most famous.

# Principal Component Analysis

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

# Create or load your dataset
# For this example, let's assume you have a DataFrame named 'df' with numeric columns

# Standardize the data (optional but recommended for PCA)
from sklearn.preprocessing import StandardScaler

# Assuming you have a DataFrame named 'df' with numeric columns
# Extract the numeric data from the DataFrame
numeric_data = top_bottom_6.select_dtypes(include=[np.number])
numeric_data=numeric_data[['Total_Att','Total_Cmp%', 'short_Cmp%','medium_Cmp%', 'long_Cmp%']]
# Standardize the data (mean=0, std=1)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numeric_data)

# Apply PCA
n_components = 2  # Number of principal components to keep
pca = PCA(n_components=n_components)
principal_components = pca.fit_transform(scaled_data)

# Create a DataFrame to store the principal components
principal_df = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2'],index=top_bottom_6['Squad'])  # Update column names accordingly
principal_df

In [ ]:
# add squad and league position to this data
principal_df=principal_df.reset_index()
x=top_bottom_6[['Squad','LgRk']]
principal_df = pd.merge(principal_df, x, on='Squad',how='left')
principal_df

In [ ]:
# Create a scatterplot with labels based on 'Squad'
plt.figure(figsize=(15, 8))
scatterplot = sns.scatterplot(data=principal_df, x='PC1', y='PC2', hue='LgRk')

# Annotate the points with squad labels
for line in range(0, principal_df.shape[0]):
    scatterplot.text(principal_df['PC1'][line]+0.01, principal_df['PC2'][line], 
                     principal_df['Squad'][line], horizontalalignment='left', 
                     size='medium', color='black', weight='semibold')

plt.title('PCA: Principal Component 1 vs Principal Component 2 with Squad Labels')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.savefig('pcascatterplot.png', dpi=300, bbox_inches='tight')
plt.show()


- As you can observe from the graph, The position of Inter Milan goalkeeper Onana is closer to Treble winner Manchester city goalkeeper Ederson. we obtain a clear understanding of how the ball-playing ability of goalkeepers can influence the positions of clubs in the top 6 and bottom 6, ultimately impacting team performance. However, there appears to be a mix-up in the above plot, which we will now investigate.

## Try pca on Principle axis 1

In [ ]:
# Create a scatterplot using only PC1
plt.figure(figsize=(10, 20))
scatterplot = plt.scatter(np.zeros_like(principal_df['PC1']),principal_df['PC1'], alpha=0.5)

# Annotate the points with squad labels
for line in range(0, principal_df.shape[0]):
    plt.text( 0,principal_df['PC1'][line]+0.01, 
             principal_df['Squad'][line], horizontalalignment='left', 
             size='medium', color='black', weight='semibold')

plt.title('Scatterplot of Principal Component 1 with Squad Labels')
plt.xlabel('Principal Component 1')
plt.yticks([])  # Remove the y-axis ticks and labels
plt.show()

## K-means clustering
- Now, let's delve into applying K-means clustering to our PCA-transformed dataframe to distinguish between top and bottom clubs based on the ball-playing ability of goalkeepers.

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix

# Assuming you have a DataFrame named 'principal_df' with columns 'Squad', 'LgRk', 'PC1', and 'PC2'

# Create a mapping between the original 'LgRk' values and cluster labels
cluster_mapping = {
    'Top6': 1,     # Assign a cluster label (e.g., 1) for the 'top6' group
    'Bottom6': 0   # Assign another cluster label (e.g., 0) for the 'bottom6' group
}

# Map the 'LgRk' values to cluster labels
principal_df['Cluster'] = principal_df['LgRk'].map(cluster_mapping)

# Perform k-means clustering with the known number of clusters (2 in this case)
kmeans = KMeans(n_clusters=2, random_state=42)
X = principal_df[['PC1', 'PC2']]
principal_df['PredictedCluster'] = kmeans.fit_predict(X)
principal_df


In [ ]:
# Create a scatterplot with different colors for each cluster
plt.figure(figsize=(10, 6))
sns.scatterplot(data=principal_df, x='PC1', y='PC2', hue='Cluster', palette='viridis', s=50)

plt.xlabel('Principal Component 1 (PC1)')
plt.ylabel('Principal Component 2 (PC2)')
plt.title('K-Means Clustering')
plt.legend(title='Cluster')
plt.savefig('kmeanscatterplot.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import ListedColormap

# Step 2: Create a scatterplot with labels based on 'Cluster'
plt.figure(figsize=(15, 8))
scatterplot = sns.scatterplot(x=principal_df['PC1'], y=principal_df['PC2'], hue=principal_df['Cluster'], palette='viridis')

# Step 3: Create a mesh grid for background colors
x_min, x_max = principal_df['PC1'].min() - 1, principal_df['PC1'].max() + 1
y_min, y_max = principal_df['PC2'].min() - 1, principal_df['PC2'].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.01), np.arange(y_min, y_max, 0.01))
Z = kmeans.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

# Step 4: Create a ListedColormap for background colors
cmap_background = ListedColormap(sns.color_palette("Set3"))

# Step 5: Plot the background colors using pcolormesh
plt.pcolormesh(xx, yy, Z, cmap=cmap_background, alpha=0.1)

# Step 6: Annotate the points with squad labels
for line in range(0, principal_df.shape[0]):
    scatterplot.text(principal_df['PC1'][line] + 0.01, principal_df['PC2'][line], 
                     principal_df['Squad'][line], horizontalalignment='left', 
                     size='medium', color='black', weight='semibold')

# Step 7: Set plot titles and labels
plt.title('PCA: Principal Component 1 vs Principal Component 2 with Cluster Labels')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
# After creating the scatterplot
plt.savefig('scatterplot.png', dpi=300, bbox_inches='tight')

plt.show()


## lets investigate how accurate is this?

In [ ]:
# Create a confusion matrix to compare the original 'Cluster' with 'PredictedCluster'
confusion = confusion_matrix(principal_df['Cluster'], principal_df['PredictedCluster'])

# Print the confusion matrix
print("Confusion Matrix:")
print(confusion)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming you have a confusion matrix 'conf_matrix' (a NumPy array)
# Replace 'conf_matrix' with your actual confusion matrix

# Create a figure
plt.figure(figsize=(8, 6))

# Create a heatmap of the confusion matrix
plt.imshow(confusion, interpolation='nearest', cmap=plt.cm.plasma)

# Add labels and annotations
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted')
plt.ylabel('Actual')

# Optionally, add tick marks and labels for classes
num_classes = len(confusion)
plt.xticks(np.arange(num_classes))
plt.yticks(np.arange(num_classes))

# Loop over data dimensions and create text annotations
for i in range(num_classes):
    for j in range(num_classes):
        plt.text(j, i, str(confusion[i, j]), horizontalalignment="center", color="white")

# Save the confusion matrix as an image
plt.savefig('confusion_matrix.png')

# Show the confusion matrix (optional)
plt.show()


### What are the instances of false positives and false negatives in this context?

## False Positives: Goalkeepers from the bottom 6 predicted as top 6 clubs:
- Sampdoria goalkeeper moved from the bottom 6 to Inter, a top 6 club.
- Stuttgart goalkeeper transitioned from the bottom 6 to SC Freiburg, another top 6 club.
- Stefan Ortega of Arminia, previously in the bottom 6, made a surprising move to Manchester City.
- Hertha BSC goalkeeper, previously in the bottom 6, transferred to Union Berlin, a top 6 club.
- Alavés goalkeeper got relegated.

## False Negatives: Goalkeepers from top 6 predicted as bottom 6 clubs:
- Betis goalkeeper, originally part of the bottom 6, remained with Betis (t6).
- Andreas Luthe moved from Union Berlin (top 6) to a second-tier club.
- Athletic Madrid, Arsenal, and Manchester United.

- As evident from the confusion matrix, false positives (where goalkeepers from the bottom 6 were predicted as top 6 clubs) saw these goalkeepers move to top 6 clubs in the subsequent season. Conversely, false negatives (where goalkeepers from the top 6 were predicted as bottom 6 clubs) included instances such as Union Berlin's goalkeeper being sold to a second division club, Manchester United sold David De Gea, and transfer rumors surrounding Arsenal's goalkeeper despite their solid performance last season. One noteworthy outlier is Athletic Madrid's goalkeeper, Jan Oblak. Athletic Madrid, known for their defensive strategy, might explain Oblak's presence on this list.

# conclusion
- It's evident that the extreme version of a modern ball-playing goalkeeper, like Onana, has the potential to make a significant impact on a club's performance. We've seen instances where goalkeepers with exceptional passing abilities have helped their teams achieve top positions in the leagues. Considering this, a move to Manchester United for a goalkeeper of Onana's caliber could be a game-changer for the club, potentially elevating their performance to new heights.